In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
# URL do site
url = "https://g1.globo.com/"

# Fazendo a requisição HTTP
response = requests.get(url)

In [ ]:
# Função para buscar o conteúdo de um link
def fetch_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Erro ao acessar {url}. Código de status: {response.status_code}")
        return None

In [ ]:
def extract_links_from_page(page_url):
    response = requests.get(page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        articles_full_section = soup.find('div', class_="_evt")
        if articles_full_section:
            links = articles_full_section.find_all('a')
            return [link.get('href') for link in links]
    else:
        print(f"Erro ao acessar {page_url}. Código de status: {response.status_code}")
        return []

In [ ]:
# Função para salvar os dados em um arquivo CSV
def save_to_csv(data, filename='fato_ou_fake-Fato.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Título', 'Textos', 'É Fake'])
        for item in data:
            writer.writerow(item)

In [ ]:
def scrape_fato_ou_fake():
    base_url = "https://g1.globo.com/index/feed/pagina-{}.ghtml"
    all_data = []
    processed_links = set()  # Conjunto para armazenar links já processados

    for page_num in range(1, 250):
        page_url = base_url.format(page_num)
        print(f"Acessando página: {page_url}")
        links = extract_links_from_page(page_url)

        for link_url in links:
            # Ignora links vazios
            if not link_url:
                print("Link vazio encontrado. Ignorando...")
                continue

            # Ignora links duplicados
            if link_url in processed_links:
                print(f"Link duplicado encontrado: {link_url}. Ignorando...")
                continue

            # Adiciona o link ao conjunto de links processados
            processed_links.add(link_url)

            print(f"\nAcessando o link: {link_url}")
            content = fetch_content(link_url)
            if content:
                link_soup = BeautifulSoup(content, 'html.parser')
                title = link_soup.find('h1', class_='content-head__title')
                title_text = title.text.strip() if title else "Título não encontrado"

                # Descarta o artigo se o título contiver "É #FAKE"
                if "É #FAKE" in title_text:
                    print(f"Artigo descartado: {title_text}")
                    continue

                # Extrai o texto de todos os parágrafos com a classe 'content-text__container'
                paragraphs = link_soup.find_all('p', class_='content-text__container')
                article_text = "\n".join([p.text.strip() for p in paragraphs]) if paragraphs else "Corpo do artigo não encontrado"

                # Adiciona os dados à lista com a flag "É Fake" = 0
                all_data.append([title_text, article_text, 0])

    save_to_csv(all_data)
    print("Dados salvos em 'fato_ou_fake.csv'")

In [ ]:
if __name__ == "__main__":
    scrape_fato_ou_fake()

Acessando página: https://g1.globo.com/index/feed/pagina-1.ghtml

Acessando o link: https://g1.globo.com/politica/noticia/2025/02/19/pena-maxima-de-2-anos-e-protecao-para-a-familia-os-premios-pedidos-por-mauro-cid-na-delacao.ghtml
Link duplicado encontrado: https://g1.globo.com/politica/noticia/2025/02/19/pena-maxima-de-2-anos-e-protecao-para-a-familia-os-premios-pedidos-por-mauro-cid-na-delacao.ghtml. Ignorando...

Acessando o link: https://g1.globo.com/politica/noticia/2025/02/19/moraes-notifica-defesa-de-denunciados-pela-pgr-e-levanta-sigilo-da-delacao-de-mauro-cid.ghtml

Acessando o link: https://g1.globo.com/politica/blog/andreia-sadi/post/2025/02/19/mauro-cid-contou-em-delacao-que-flavio-bolsonaro-eduardo.ghtml
Link duplicado encontrado: https://g1.globo.com/politica/blog/andreia-sadi/post/2025/02/19/mauro-cid-contou-em-delacao-que-flavio-bolsonaro-eduardo.ghtml. Ignorando...

Acessando o link: https://g1.globo.com/politica/blog/andreia-sadi/post/2025/02/19/grupo-moderado-temia-q